## Preprocess

수집한 데이터를 전처리하여 다시 저장합니다.

In [2]:
import pandas


def load(list_num: int) -> pandas.DataFrame:
    return pandas.read_csv(f'list{list_num}.csv')

In [3]:
# 어떻게 생겼는지 가져와서 일단 봅니다.
load(1)
#load(2)
#load(3)

,번호,구분,계약명,계약금액,계약일,계약상대자
0,1,공사,"바이오,해양클러스터관 건립 관급자재 구매(데크플레이트) 계약의뢰",120012920,2021-12-03,주식회사 디앤에프
1,2,자산,[예술체육대학 디자인학부]교육 연구용 기자재(imac) 구매,63002500,2021-11-22,주식회사 빛나는교육
2,3,용역,대학발전계획 수립 컨설팅 용역 의뢰,291059000,2021-11-19,이와이컨설팅 유한책임회사
3,4,공사,산학 창업 COMPLEX(바이오 해양 클러스터관) 건립 전기공사 계약(2021년 연...,16742900,2021-11-16,주식회사 우일
4,5,공사,"바이오,해양클러스터관 건립공사(건축분야) 계약의뢰",502860000,2021-11-15,고속건설(주)
...,...,...,...,...,...,...
1074,1075,용역,(총무과)버스 임차계약 의뢰,3060000,2013-01-23,(주)이삼화관광
1075,1076,용역,(총무과)2013년 인천대학교 교직원 단체보험 입찰의뢰,50891350,2013-01-18,한국교직원공제회
1076,1077,용역,인천대학교 차세대 ui종합정보시스템 1단계 구축 감리용역,82500000,2012-11-27,(주)케이씨에이
1077,1078,자산,차세대 ui 종합정보시스템 1단계 구축사업,857520000,2012-08-10,(주)엔디에스


## 전처리 목표: 구매 항목을 분리해내기

### 주어진 데이터셋 관찰

총 32079개의 레코드를 가져왔습니다.

데이터셋은 다섯 개의 칼럼을 제공합니다: `구분`, `계약명`, `계약금액`, `계약금액`, `계약일`, `계약대상자`.

구매 내역은 그 중 `계약명`에 대체로 자유로운 형태로 기술되어 있습니다.

우리에게 필요한 것은 *{**어떤**} 물건을 구매하였는가* 에서 *{**어떤**}* 부분입니다.

### 구매 내역이 아닌 것을 제거: 주어진 칼럼 `구분`을 사용

주어진 데이터셋은 물품 구입 뿐만 아니라 공사 발주도 포함합니다:

```
0	1	공사	바이오,해양클러스터관 건립 관급자재 구매(데크플레이트) 계약의뢰	120012920	2021-12-03	주식회사 디앤에프
```

이 경우는 `구분`이 `공사`인 것을 제하는 것으로 해결할 수 있습니다. 비슷한 경우로 `용역`도 제거합니다. 남은 것은 무엇이 있을까요?

```
0	공사
1	자산
2	용역
19	구매/기타
```

데이터셋 전체에 등장하는 구분은 위 4개가 전부입니다. 이제 `자산`과 `구매/기타`만 남았습니다. 이 둘은 일단 남겨 놓습니다.

이제 주어진 데이터셋에서 27762개의 레코드만 남았습니다.

### 구매 내역이 아닌 것을 제거: 텍스트로부터

아래와 같은 계약 내역은 어떠한 물품을 구매했는지를 제대로 알려주지 않습니다:

```
10172	10173	구매/기타	송도캠퍼스 환경미화원실 근무환경 개선작업	2438700	2017-01-04	고려설비(주)
```

`계약명`이 아래와 같은 형태의 값을 가지는 기록만 남기기로 했습니다:

```
^(?<ITEM>.+) (구매|구입|구입요구)$
```

11188개만 남았습니다.

### 학과 이름 제거

구매 항목을 살펴 보니 *예술체육대학*, *임베디드시스템학과*와 같은 학과/학부 이름이 자주 등장합니다. 이러한 이름들은 구매한 물품을 설명하는 데에 도움이 되지는 않기 때문에 제거하고 싶습니다.

[교내 홈페이지](https://www.inu.ac.kr/mbshome/mbs/inu/index.do)로부터 학과 목록을 모두 가져 옵니다.

> ['유아교육과',
 '경영학부',
 '건설환경공학',
 '예술체육대학',
 '영어영문학과',
 '컴퓨터공학부',
 '화학과',
 '공연예술학과',
 '무역학부',
 '경제학과',
 '임베디드시스템공학과',
 '체육교육과',
 '생명공학전공',
 '생명과학전공',
 '체육학부',
 '수학교육과',
 '계약학과',
 '인문대학',
 '신소재공학과',
 '일어교육과',
 '행정학과',
 '창의인재개발학과',
 '한국통상전공',
 '생명과학부',
 '운동건강학부',
 '정치외교학과',
 '공과대학',
 '국어교육과',
 '사회과학대학',
 '역사교육과',
 '경영대학',
 '동북아통상전공',
 '도시공학과',
 '영어교육과',
 '에너지화학공학과',
 '중어중국학과',
 '생명공학부',
 '나노바이오전공',
 '정보통신공학과',
 '정보기술대학',
 '신문방송학과',
 '수학과',
 '산업경영공학과',
 '물리학과',
 '글로벌정경대학',
 '디자인학부',
 '불어불문학과',
 '테크노경영학과',
 '도시건축학부',
 '패션산업학과',
 '독어독문학과',
 '사범대학',
 '분자의생명전공',
 '기초교육원',
 '세무회계학과',
 '국어국문학과',
 '생명과학기술대학',
 '윤리교육과',
 '동북아국제통상학부',
 '메카트로닉스공학',
 '법학부',
 '도시행정학과',
 '건축공학',
 '안전공학과',
 '사회복지학과',
 '조형예술학부',
 '도시환경공학부',
 '일어일문학과',
 '환경공학',
 '문헌정보학과',
 '기계공학과',
 '도시건축학',
 '도시과학대학',
 '전자공학과',
 '전기공학과',
 '해양학과',
 '자연과학대학',
 '소비자학과']

단과대와 학과가 모두 포함된 리스트입니다. 이 텍스트만 걸러내면 됩니다.

In [50]:
import re
from urllib.request import urlopen
from bs4 import BeautifulSoup


def get_list_of_departments() -> list:
    html = urlopen('https://www.inu.ac.kr/mbshome/mbs/inu/index.do')
    soup = BeautifulSoup(html, "html.parser")

    rows = [tag.text for tag in soup.select('#gnb > ul > li')[0].select('div > ul > li')]

    decoupled = map(lambda x: x.replace('(', '\n').replace(')', ''), rows)

    split = map(lambda x: x.split('\n'), decoupled)

    flatten = [x for sublist in split for x in sublist if len(x) > 0][1:]

    return list(set(flatten))

In [78]:
from konlpy.tag import Okt


def preprocess_and_save():
    combined = load(1).append(load(2)).append(load(3))

    # 용역 같은 것 제거
    category_filtered = combined[(combined['구분'] == '자산') | (combined['구분'] == '구매/기타')]

    # 특정 형식 텍스트를 지정하는 패턴
    purchase_regex = re.compile(r'^(?P<ITEM>.+) (구매|구입|구입요구)$')

    # 특정 형식 텍스트만 추출
    purchase_filtered = category_filtered[category_filtered['계약명'].str.match(purchase_regex)].copy()

    # 그렇게 추출된 물품 이름을 새 칼럼에 할당
    purchase_filtered['물품'] = purchase_filtered['계약명'].map(lambda x: re.match(purchase_regex, x).group('ITEM'))

    okt = Okt()

    # 단과대학과 학과명은 제거할 예정
    deps = get_list_of_departments()

    # 단과대학과 학과명을 걸러내는 큰 패턴
    big_deps_regex = re.compile('|'.join(map(re.escape, deps)))

    # 단과대학과 학과명을 걸러낸 시리즈
    without_deps = purchase_filtered['물품'].map(lambda x: big_deps_regex.sub('', x))

    # 명사만 추출
    all_nouns = without_deps.map(lambda x: okt.nouns(x))

    # 명사 중에서도 길이가 2 이상인 것만 추출
    short_nouns_filtered = all_nouns.map(lambda x: [k for k in x if len(k) > 1])

    # 위에서 추출한 시리즈를 다시 데이터프레임에 대입
    purchase_filtered['토큰'] = short_nouns_filtered

    # 사용할 칼럼만 추출
    only_necessary_columns = purchase_filtered.filter(['물품', '토큰', '계약금액', '계약일'])\

    # 계약일 오름차순 정렬
    date_sorted = only_necessary_columns.sort_values(by=['계약일'], axis=0)

    # 파일로 저장
    date_sorted.to_csv('preprocessed.csv', index=False)

In [79]:
preprocess_and_save()